In [1]:
import tensorflow as tf
import datetime
import numpy as np
import matplotlib.pyplot as plt
import io
import sklearn.metrics
import itertools
from six.moves import range
from tensorflow import keras

In [2]:
print(tf.__version__)

2.0.0-beta1


In [3]:
# Load the fashion-mnist pre-shuffled train data and test data
(x_train, y_train), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

In [4]:
# Define the text labels
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [5]:
x_train = x_train / 255.0
test_images = test_images / 255.0

In [6]:
# Further break training data into train / validation sets (# put 5000 into validation set and keep remaining 55,000 for train)
(x_train, valid_images) = x_train[5000:], x_train[:5000] 
(y_train, valid_labels) = y_train[5000:], y_train[:5000]

# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
valid_images = valid_images.reshape(valid_images.shape[0], w, h, 1)
test_images = test_images.reshape(test_images.shape[0], w, h, 1)

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
valid_labels = tf.keras.utils.to_categorical(valid_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)


In [7]:
from tensorflow.keras import layers,models
#开始建立模型啦
model = tf.keras.Sequential()

model.add(layers.Conv2D(32, kernel_size=(3, 3),
                      activation='relu',
                      input_shape=[28, 28, 1]))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(3, 3)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(3, 3)))

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))

model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 4096)              5

In [9]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [10]:
#clear any logs from previous runs
!rm -rf ./logs

In [11]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)
model.fit(x_train,
         y_train,
         epochs=50,
         validation_data=(valid_images, valid_labels),
         callbacks=[tensorboard_callback])

W0713 02:04:04.248782 140476427331328 deprecation.py:323] From /home/zelda/anaconda3/envs/tensor/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 55000 samples, validate on 5000 samples
Epoch 1/50
55000/55000 [==============================] - 233s 4ms/sample - loss: 0.4883 - accuracy: 0.8200 - val_loss: 0.3262 - val_accuracy: 0.8846
Epoch 2/50
55000/55000 [==============================] - 208s 4ms/sample - loss: 0.3107 - accuracy: 0.8861 - val_loss: 0.2844 - val_accuracy: 0.8962
Epoch 3/50
55000/55000 [==============================] - 189s 3ms/sample - loss: 0.2663 - accuracy: 0.9025 - val_loss: 0.2665 - val_accuracy: 0.9066
Epoch 4/50
55000/55000 [==============================] - 182s 3ms/sample - loss: 0.2402 - accuracy: 0.9112 - val_loss: 0.2453 - val_accuracy: 0.9098
Epoch 5/50
55000/55000 [==============================] - 236s 4ms/sample - loss: 0.2204 - accuracy: 0.9197 - val_loss: 0.2590 - val_accuracy: 0.9072
Epoch 6/50
55000/55000 [==============================] - 309s 6ms/sample - loss: 0.2048 - accuracy: 0.9247 - val_loss: 0.2315 - val_accuracy: 0.9170
Epoch 7/50
55000/55000 [===========================

In [13]:

test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

10000/10000 [==============================] - 10s 1ms/sample - loss: 0.5112 - accuracy: 0.9139
Test accuracy: 0.9139
